In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [22]:
# have 2 rectangular sections of Pu_239 and Ca_40 (testing fission and n, alpha cross-sections) (cross-sections sourced from ENDF at 1 MeV)
sig_fPu = 772.71246e-24 # cm^2, Pu is in thermal cross-sections
sig_aPu = 278.0934e-24 + 772.71246e-24 # cm^2; includes all non-scattering collisions
sig_tPu = 1058.8896e-24
rho_Pu = 19.84 # g/cc

sig_alphCa = 0.002576135e-24 # cm^2, Pu is in thermal cross-sections
sig_aCa = 0.4256061e-24 + 0.002576135e-24 # cm^2
sig_tCa = 3.0945628e-24
rho_Ca = 1.526 # g/cc
perc_Ca40 = .969


def MacroXS(rho, A, sig, perc=1):
    return rho*6.022e23/239 * sig * perc

shield = [(MacroXS(rho_Pu, 239, sig_tPu), MacroXS(rho_Pu, 239, sig_aPu),MacroXS(rho_Pu, 239, sig_fPu), .1), # plutonium
        (MacroXS(rho_Ca, 239, sig_tCa, perc_Ca40), MacroXS(rho_Ca, 239, sig_aCa, perc_Ca40),MacroXS(rho_Ca, 239, sig_alphCa, perc_Ca40), .1)
        ] # an array of Sigma_t's, Sigma_a's, Sigma_f/Sigma_alpha's and thicknesses

def slab_rxn_basic(Nparticles):
    transmitted, reflected, absorbed, spec_abs = 0, 0, 0, 0
    for _ in range(int(Nparticles)):
        x, mu, region = 0.0, 1.0, 0
        while True:
            Sig_t, Sig_a, Sig_Special, thickness = shield[region]
            l = -np.log(1-np.random.random())/Sig_t
            x += l*mu
            
            if 0 <= x < thickness:  # scatter or absorption
                if np.random.random() < Sig_a/Sig_t:
                    absorbed += 1
                    if np.random.random() < Sig_Special/Sig_a:
                        spec_abs += 1
                    break
                mu = np.random.uniform(-1,1)  # pick new mu
            elif x >= thickness:  # go to next region
                if region == len(shield)-1: # check if fully through shield
                    transmitted += 1
                    break
                region += 1
                x = 0.0
            else:  # Backward
                if region == 0:
                    reflected += 1
                    break
                region -= 1
                x = thickness-x # start particle where it would have scattered back to in previous region
    return transmitted, reflected, absorbed, spec_abs

def slab_rxn_implicit(Nparticles, weight_cutoff=.25, weight_survival=1):
    transmitted, reflected, absorbed = 0,0, 0
    for i in range(int(Nparticles)):
        x, mu, region, weight = 0.0, 1.0, 0, 1.0
        while weight > 0:
            Sig_t, Sig_a, Sig_Special, thickness = shield[region]
            # Implicit capture
            absorption_prob = Sig_a/Sig_t
            weight *= (1 - absorption_prob)          
            l = -np.log(1-np.random.random())/Sig_t
            x += l*mu

            # Russian roulette
            if weight < weight_cutoff:
                if np.random.random() < (1-weight/weight_survival): # kill the particle
                    break
                else:
                    weight=weight_survival
            
            if 0 <= x < thickness:  # Scatter
                mu = np.random.uniform(-1,1)
            elif x >= thickness:  # escape
                if region == len(shield)-1:
                    transmitted += weight
                    break
                region += 1
                x = 0.0
            else:  # reflection
                if region == 0:
                    reflected += weight
                    break
                region -= 1
                x = thickness-x     
    absorbed = Nparticles - transmitted - reflected
    spec_abs = absorbed * (Sig_Special/Sig_a)
    return transmitted, reflected, absorbed, spec_abs

In [21]:
basic = slab_rxn_basic(10000)
implicit = slab_rxn_implicit(10000)

print(f"With the basic solver, the probability that a particle is transmitted is {basic[0]}, reflected is {basic[1]}, and absorbed {basic[2]}. Special absorptions are {basic[3]}.")
print(f"With implicit capture, the probability that a particle is transmitted is {implicit[0]}, reflected is {implicit[1]}, and absorbed {implicit[2]}. Special absorptions are {implicit[3]}.")

With the basic solver, the probability that a particle is transmitted is 55, reflected is 15, and absorbed 9930. Special absorptions are 7359.
With implicit capture, the probability that a particle is transmitted is 0, reflected is 1, and absorbed 9999. Special absorptions are 7352.787209941901.


In [28]:
# in order to simulate layers and their specific reactions better as well as to actually simulate the conditions in 
shield1 = [MacroXS(rho_Pu, 239, sig_tPu), MacroXS(rho_Pu, 239, sig_aPu),MacroXS(rho_Pu, 239, sig_fPu), .1]
shield2 = [MacroXS(rho_Ca, 239, sig_tCa, perc_Ca40), MacroXS(rho_Ca, 239, sig_aCa, perc_Ca40),MacroXS(rho_Ca, 239, sig_alphCa, perc_Ca40), .1]

def slab_rxn_implicit(Nparticles, shield, weight_cutoff=.25, weight_survival=1):
    transmitted, reflected, absorbed = 0,0, 0
    for i in range(int(Nparticles)):
        x, mu, weight = 0.0, np.random.random(), 1.0 # because there's a point source in the middle of the 2 layers, the scattering angle on either side will be from [0,1] 
        while weight > 0:
            Sig_t, Sig_a, Sig_Special, thickness = shield
            # Implicit capture
            absorption_prob = Sig_a/Sig_t
            weight *= (1 - absorption_prob)          
            l = -np.log(1-np.random.random())/Sig_t
            x += l*mu

            # Russian roulette
            if weight < weight_cutoff:
                if np.random.random() < (1-weight/weight_survival): # kill the particle
                    break
                else:
                    weight=weight_survival
            
            if 0 <= x < thickness:  # Scatter
                mu = np.random.uniform(-1,1)
            elif x >= thickness:  # escape
                transmitted += weight
                break
            else:  # reflection
                reflected += weight
                break  
    absorbed = Nparticles - transmitted - reflected
    spec_abs = absorbed * (Sig_Special/Sig_a)
    return transmitted, reflected, absorbed, spec_abs

In [32]:
# Testing 20000 neutrons from the source, split in half
Pu_implicit = slab_rxn_implicit(10000, shield1)
Ca_implicit = slab_rxn_implicit(10000, shield2)
print(f"For the plutonium layer, the probability that a particle is transmitted is {int(Pu_implicit[0])/10000}, reflected is {int(Pu_implicit[1])/10000}, and absorbed {int(Pu_implicit[2])/10000}. Fissions are around {int(Pu_implicit[3])/10000}.")
print(f"For the calcium layer, the probability that a particle is transmitted is {int(Ca_implicit[0])/10000}, reflected is {int(Ca_implicit[1])/10000}, and absorbed {int(Ca_implicit[2])/10000}. (n,alpha) reactions are around {int(Ca_implicit[3])/10000}.")


For the plutonium layer, the probability that a particle is transmitted is 0.0, reflected is 0.0001, and absorbed 0.9999. Fissions are around 0.7352.
For the calcium layer, the probability that a particle is transmitted is 0.8577, reflected is 0.0028, and absorbed 0.1394. (n,alpha) reactions are around 0.0008.


This is an incredibly simplisitic view because it does not take into account how fissions create new neutrons, only treats slabs as 1-dimensional, reflections don't reflect to the other slab 